In [ ]:

#r "nuget:Microsoft.DotNet.Interactive.SqlServer,1.0.0-beta.22175.2"
#r "nuget:Microsoft.DotNet.Interactive,1.0.0-beta.22175.2"
#r "nuget:Microsoft.DotNet.Interactive.PowerShell,1.0.0-beta.22175.2"
#r "nuget:Microsoft.DotNet.Interactive.Http,1.0.0-beta.22175.2"

#r "nuget:FSharp.Core, *-*"
#r "nuget:Plotly.NET,*-*"
#r "nuget: Plotly.NET.Interactive, *-*"
using Plotly.NET;

In [ ]:
#!connect mssql --kernel-name splSnapshots "Server=localhost;Database=SeattlePublicLibraryOpenData;Trusted_Connection=True;TrustServerCertificate=true;"

In [ ]:
<style>
pre {
    overflow-x: scroll
}
</style>

In [ ]:
#!sql-splSnapshots --name itemTypeAgeGroup


;WITH ageGroup_cte
AS (SELECT
  ReportDate,
  AgeGroup,
  COUNT(DISTINCT bibnum) AS ItemCount,
  SUM(ei.itemcount) AS InventoryCount

FROM ElementInventory ei
INNER JOIN ItemType it
  ON ei.ItemTypeCode = it.Code
WHERE ei.ItemCount > 0
GROUP BY ReportDate,
         AgeGroup),
calculatedReportDateTotal_cte
AS (SELECT
  ReportDate,
  SUM(itemcount) AS itemCountSum,
  SUM(inventoryCount) AS inventoryCountSum
FROM ageGroup_cte
GROUP BY ReportDate)
SELECT
  r.ReportDate,
  AgeGroup,
  CAST(ItemCount AS float) / itemCountSum * 100 AS AgeGroupItemPercentage,
  CAST(InventoryCount AS float) / inventoryCountSum * 100 AS AgeGroupInventoryPercentage

FROM calculatedReportDateTotal_cte r
INNER JOIN ageGroup_cte a
  ON a.ReportDate = r.ReportDate

ORDER BY ReportDate, AgeGroup




In [ ]:
#!share --from sql-splSnapshots itemTypeAgeGroup

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;

string[] years = (itemTypeAgeGroup[0].Data.GroupBy(x=>((DateTime)x["ReportDate"]).Year).OrderBy(x=>x.Key.ToString())
.SelectMany(x=>new string[]{x.Key.ToString(), x.Key.ToString()})).ToArray();

string[] inventoryCategory = (Enumerable.Repeat(new string[]{"Item Percentage", "Inventory Percentage"}, years.Length/2).SelectMany(x=>x)).ToArray();
List<Trace> traces = new();

foreach(var g in itemTypeAgeGroup[0].Data.GroupBy(x=> x["AgeGroup"]).OrderBy(x=>x.Key.ToString())){

    var itemPercentages = g.GroupBy(x=>((DateTime)x["ReportDate"]).Year).Select(x=> x.Average(x=>(double)x["AgeGroupItemPercentage"]));
    var inventoryPercentages = g.GroupBy(x=>((DateTime)x["ReportDate"]).Year).Select(x=> x.Average(x=>(double)x["AgeGroupInventoryPercentage"]));

    Trace t = new Trace("bar");
    t.SetValue("x", new string[][]{years, inventoryCategory} );
    t.SetValue("y", itemPercentages.Zip(inventoryPercentages, (x,y) => new double[]{x,y}).SelectMany(x=> x));
    t.SetValue("name", g.Key.ToString());
    traces.Add(t);
}
var layout = Layout.init<int>(
    Title: Title.init("Age Group Composition %"),
    Width:1500,
    DragMode:StyleParam.DragMode.Pan,
    BarMode:StyleParam.BarMode.Stack
    ,    Colorway: Color.fromColors(
        new List<Color>() { Color.fromKeyword(ColorKeyword.CornflowerBlue), Color.fromKeyword(ColorKeyword.SeaGreen)})
);

#!html
<a id="ageGroup"></a>


#!csharp

GenericChart.combine(new GenericChart.GenericChart[]{GenericChart.ofTraceObject(true, traces[0]), GenericChart.ofTraceObject(true, traces[1])})
.WithLayout(layout)

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#!sql-splSnapshots --name itemTypeFormat

;WITH formatGroup_cte
AS (SELECT ReportDate,
       FormatGroup,
       Count(DISTINCT bibnum) as ItemCount,
       Sum(ei.itemcount) as InventoryCount
FROM   ElementInventory ei
       INNER JOIN ItemType it
               ON ei.ItemTypeCode = it.Code
WHERE ei.ItemCount > 0
GROUP  BY ReportDate,
          FormatGroup),
calculatedReportDateTotal_cte
AS (SELECT
  ReportDate,
  SUM(itemcount) AS itemCountSum,
  SUM(inventoryCount) AS inventoryCountSum
FROM formatGroup_cte
GROUP BY ReportDate)
SELECT
  r.ReportDate,
  FormatGroup,
  CAST(ItemCount AS float) / itemCountSum * 100 AS FormatGroupItemPercentage,
  CAST(InventoryCount AS float) / inventoryCountSum * 100 AS FormatGroupInventoryPercentage

FROM calculatedReportDateTotal_cte r
INNER JOIN FormatGroup_cte a
  ON a.ReportDate = r.ReportDate

ORDER BY ReportDate, FormatGroup




In [ ]:
#!share --from sql-splSnapshots itemTypeFormat

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;


string[] years = (itemTypeFormat[0].Data.GroupBy(x=>((DateTime)x["ReportDate"]).Year).OrderBy(x=>x.Key.ToString())
.SelectMany(x=>new string[]{x.Key.ToString(), x.Key.ToString()})).ToArray();

string[] inventoryCategory = (Enumerable.Repeat(new string[]{"Item Percentage", "Inventory Percentage"}, years.Length/2).SelectMany(x=>x)).ToArray();
List<Trace> traces = new();

foreach(var g in itemTypeFormat[0].Data.GroupBy(x=> x["FormatGroup"]).OrderBy(x=>x.Key.ToString())){

    var itemPercentages = g.GroupBy(x=>((DateTime)x["ReportDate"]).Year).Select(x=> x.Average(x=>(double)x["FormatGroupItemPercentage"]));
    var inventoryPercentages = g.GroupBy(x=>((DateTime)x["ReportDate"]).Year).Select(x=> x.Average(x=>(double)x["FormatGroupInventoryPercentage"]));

    Trace t = new Trace("bar");
    t.SetValue("x", new string[][]{years, inventoryCategory} );
    t.SetValue("y", itemPercentages.Zip(inventoryPercentages, (x,y) => new double[]{x,y}).SelectMany(x=> x));
    t.SetValue("name", g.Key.ToString());
    traces.Add(t);
}
var layout = Layout.init<int>(
    Title: Title.init("Format Group Composition %"),
    Width:1500,
    DragMode:StyleParam.DragMode.Pan,
    BarMode:StyleParam.BarMode.Stack
    ,    Colorway: Color.fromColors(
        new List<Color>() { Color.fromKeyword(ColorKeyword.DarkTurquoise), Color.fromKeyword(ColorKeyword.CornflowerBlue), Color.fromKeyword(ColorKeyword.PaleVioletRed), Color.fromKeyword(ColorKeyword.GoldenRod)})
);

#!html
<a id="formatGroup"></a>


#!csharp

GenericChart.combine(traces.Select(x=> GenericChart.ofTraceObject(true,x)))
.WithLayout(layout)

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

the item type 'other' was only briefly used from 2017 to 2019.  77 Titles were see to have other at one point during this time.  Then that type was no longer used and items were switch to 'print' item type

In [ ]:
#!sql-splSnapshots --name longRunningPeak

;WITH longrunningpeak_cte
     AS (SELECT ed.bibnum,
                Min(ed.reportdate)                                      AS
                minPeak,
                Max(ed.reportdate)                                      AS
                maxPeak,
                Datediff(month, Min(ed.reportdate), Max(ed.reportdate)) AS
                amountPeak
         FROM   elementdetail ed
                INNER JOIN elementinventory ei
                        ON ei.bibnum = ed.bibnum
                           AND ei.reportdate = ed.reportdate
                INNER JOIN itemtype it
                        ON ei.itemtypecode = it.code
         WHERE  it.code = 'pkbknh'
         GROUP  BY ed.bibnum)
SELECT TOP 10 lrp.*,
              t.title
FROM   longrunningpeak_cte lrp
       INNER JOIN (SELECT bibnum,
                          Max(titleid) AS TitleId
                   FROM   elementdetail
                   GROUP  BY bibnum) ed
               ON ed.bibnum = lrp.bibnum
       INNER JOIN title t
               ON t.titleid = ed.titleid
ORDER  BY amountpeak DESC 

In [ ]:
#!share --from sql-splSnapshots longRunningPeak
using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;
using System;
using Range = Plotly.NET.StyleParam.Range;
using ChartDomain = Plotly.NET.ChartDomain.Chart;

var header =  new string[]{"Title", "Months as Peak Pick"};
var peaksTable = ChartDomain.Table<string, char, IEnumerable<string>,string>(
    header
    ,longRunningPeak[0].Data.Select(x => new string[]{ x["title"].ToString(), x["amountPeak"].ToString()})
    ,MultiColumnWidth: new double[]{.85,.15}
    ,CellsAlign: StyleParam.HorizontalAlign.Left
    
).WithLayout(
    Layout.init<int>(
        Width:770,
        Height:235,
        Margin:Margin.init<int,int,int,int,int,int>(Top:10, Left: 10, Right: 10, Bottom:10)
));


#!html
<a id="longRunningPeaks"></a>

#!csharp
peaksTable

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#!sql-splSnapshots --name trumpAuthors


;with auth_cte as(
    select * from author where (Name like '%carlson%' and name like '%tucker%')
    or (Name like '%ted%' and name like '%Cruz%')
    OR (Name like '%donald%' and name like '%trump%')
    ),
    authInventorySum_cte as (
    select ei.ReportDate, ac.Name, sum(ei.itemcount) as authoritemcount
    from ElementInventory ei
    inner join ElementDetail ed on ed.BibNum = ei.BibNum and ed.ReportDate = ei.ReportDate
    inner join auth_cte ac on ac.authorid = ed.AuthorId
    group by ei.reportdate, ac.Name
    )
    select * from authInventorySum_cte
    order by ReportDate

In [ ]:
#!share --from sql-splSnapshots trumpAuthors
using System;
using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;
using Range = Plotly.NET.StyleParam.Range;
using static Plotly.NET.StyleParam;


List<GenericChart.GenericChart> charts = new();
var annotation = new Annotation();
int chartIndx = 1;
foreach(var g in trumpAuthors[0].Data.GroupBy(x=>x["Name"])){
    annotation = new Annotation();
    annotation.SetValue("text", g.Key.ToString());
    annotation.SetValue("showarrow", false);
    annotation.SetValue("xref", $"x{chartIndx} domain" );
    annotation.SetValue("yref", $"y{chartIndx++} domain"); //paper
    annotation.SetValue("x", .5);
    annotation.SetValue("y", 1);
    annotation.SetValue("yshift", 30);

    charts.Add(
        Chart2D.Chart.Column<string, int, string, string, string>(
            Name:g.Key.ToString()  ,
            keysValues: new List<Tuple<string,int>>(
                g.Select(kvp => new Tuple<string, int>(((DateTime)kvp["ReportDate"]).ToShortDateString(), int.Parse(kvp["authoritemcount"].ToString())))
            )   
        )
        .WithXAxisStyle(title: Title.init(),ShowGrid:false, ShowLine:true)
        .WithYAxisStyle(title: Title.init())
        .WithAnnotation(annotation)
       // .WithYAxis(LinearAxis.init<int,int,int,int,int,int>(Range:Range.NewMinMax(0,maxRange*1.05)))
    );
}

var layout = Layout.init<int>(
    Width:1250,
     Height:1550,
     DragMode:StyleParam.DragMode.Pan
     ,Grid: LayoutGrid.init(
         SubPlots:
        new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[][]
        {
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(1), StyleParam.LinearAxisId.NewY(1)) , },
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(2), StyleParam.LinearAxisId.NewY(2)) , },
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(3), StyleParam.LinearAxisId.NewY(3)), },
            new Tuple<StyleParam.LinearAxisId, StyleParam.LinearAxisId>[]{ Tuple.Create(StyleParam.LinearAxisId.NewX(4), StyleParam.LinearAxisId.NewY(4)), }
           }
        )
     ,Colorway:Color.fromColors(
         Enumerable.Repeat(Color.fromKeyword(ColorKeyword.PaleVioletRed),3)
         .Prepend(Color.fromKeyword(ColorKeyword.Orange))
         )
     ,ShowLegend:false
    );
    var g = Chart.Grid<IEnumerable<GenericChart.GenericChart>>(4,1
        ,YGap:.37    
        ,XGap:.1
        )
    .Invoke(charts)
    .WithLayout(layout
        );

#!html
<a id="panel"></a>

#!csharp
g

<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#!sql-splSnapshots --name filteredAllAuthors

; WITH authorcount_cte
AS (SELECT authorid,
           Count(*) authorCount
    FROM   elementdetail
    WHERE  reportdate = '2022-05-01'
    GROUP  BY authorid)
SELECT DISTINCT TOP (10)  acc.*,
           a.NAME
--, it.Description 
--, t.Title 
FROM   authorcount_cte acc
  INNER JOIN author a
          ON a.authorid = acc.authorid
  INNER JOIN elementdetail ed
          ON a.authorid = ed.authorid
             AND ed.reportdate = '2022-05-01'
  --inner join Title t on t.TitleId = ed.TitleId
  INNER JOIN elementinventory ei
          ON ei.bibnum = ed.bibnum
             AND ei.reportdate = '2022-05-01'
  INNER JOIN itemtype it
          ON it.code = ei.itemtypecode
WHERE  a.authorid NOT IN( 30861 --blank
                     , 264--Geological Survey (U.S.)
                    )
  AND NAME NOT LIKE ( 'United States%' )
--and name not like ('United States. Congress.%')
--and name not like ('United States. President%')
ORDER  BY authorcount DESC 

In [ ]:
#!share --from sql-splSnapshots filteredAllAuthors
using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;
using System;
using Range = Plotly.NET.StyleParam.Range;
using ChartDomain = Plotly.NET.ChartDomain.Chart;

var header =  new string[]{"Author", "Item Count"};
var authorsTable = ChartDomain.Table<string, char, IEnumerable<string>,string>(
    header
    ,filteredAllAuthors[0].Data.Select(x => new string[]{ x["NAME"].ToString(), x["authorCount"].ToString()})
    ,MultiColumnWidth: new double[]{.85,.15}
    ,CellsAlign: StyleParam.HorizontalAlign.Left
    
).WithLayout(
    Layout.init<int>(
        Width:500,
        Height:267,
        Margin:Margin.init<int,int,int,int,int,int>(Top:10, Left: 10, Right: 10, Bottom:10)
));


#!html
<a id="filteredAllAuthors"></a>

#!csharp
authorsTable



<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

The large amount items is also due to different content types.  For Bach there's sheet music, audio CD, audio records, and books
for Agatha Christie there's translations, anthologies, and different versions

In [ ]:
#!sql-splSnapshots --name topPeakInventoryCount

;WITH peak_cte
     AS (SELECT DISTINCT ed.bibnum,
                         ed.ReportDate
         FROM   elementdetail ed
                INNER JOIN elementinventory ei
                        ON ei.bibnum = ed.bibnum
                           AND ei.reportdate = ed.reportdate
                INNER JOIN itemtype it
                        ON ei.itemtypecode = it.code
         WHERE  it.code = 'pkbknh'),
     allpeakinventory_cte
     AS (SELECT ed.ReportDate,
                ed.bibnum,
                Sum(ei.ItemCount) PeakPickInventoryCount
         FROM   elementdetail ed
                INNER JOIN elementinventory ei
                        ON ei.bibnum = ed.bibnum
                           AND ei.reportdate = ed.reportdate
                INNER JOIN itemtype it
                        ON ei.itemtypecode = it.code
                INNER JOIN peak_cte pcte
                        ON pcte.BibNum = ed.BibNum
                           AND ed.ReportDate = pcte.ReportDate
         GROUP  BY ed.ReportDate,
                   ed.BibNum),
     toppeakinventorycount_cte
     AS (SELECT TOP(10) ReportDate,
                        BibNum,
                        PeakPickInventoryCount
         FROM   allpeakinventory_cte
         ORDER  BY PeakPickInventoryCount DESC,
                   ReportDate,
                   BibNum)
SELECT tpic.ReportDate,
       tpic.PeakPickInventoryCount,
       t.title
FROM   toppeakinventorycount_cte tpic
       INNER JOIN ElementDetail ed
               ON tpic.BibNum = ed.BibNum
                  AND tpic.ReportDate = ed.ReportDate
       INNER JOIN Title t
               ON t.TitleId = ed.TitleId
ORDER  BY PeakPickInventoryCount DESC,
          tpic.ReportDate,
          tpic.BibNum 


In [ ]:
#!share --from sql-splSnapshots topPeakInventoryCount

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;
using System;
using Range = Plotly.NET.StyleParam.Range;
using ChartDomain = Plotly.NET.ChartDomain.Chart;

var header =  new string[]{"Report Date", "Peak Pick Inventory Count", "Title"};
var topPeakInventoryTable = ChartDomain.Table<string, char, IEnumerable<string>,string>(
    header
    ,topPeakInventoryCount[0].Data.Select(x => new string[]{ ((DateTime)x["ReportDate"]).ToShortDateString(), x["PeakPickInventoryCount"].ToString(), x["title"].ToString()})
    ,MultiColumnWidth: new double[]{.09,.15,.45}
    ,CellsAlign: StyleParam.HorizontalAlign.Left
    
).WithLayout(
    Layout.init<int>(
        Width:700,
        Height:267,
        Margin:Margin.init<int,int,int,int,int,int>(Top:10, Left: 10, Right: 10, Bottom:10)
));


#!html
<a id="topPeakInventoryCount"></a>

#!csharp
topPeakInventoryTable



<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#!sql-splSnapshots --name topPeakAuthorCount
;WITH peak_cte
     AS (SELECT DISTINCT ed.bibnum,
                         ed.ReportDate
         FROM   elementdetail ed
                INNER JOIN elementinventory ei
                        ON ei.bibnum = ed.bibnum
                           AND ei.reportdate = ed.reportdate
                INNER JOIN itemtype it
                        ON ei.itemtypecode = it.code
         WHERE  it.code = 'pkbknh'
                AND AuthorId <> 30861),
     allpeakauthorcount_cte
     AS (SELECT ed.ReportDate,
                ed.authorid,
                Count(DISTINCT ed.BibNum) authorElementCount
         FROM   elementdetail ed
                INNER JOIN elementinventory ei
                        ON ei.bibnum = ed.bibnum
                           AND ei.reportdate = ed.reportdate
                INNER JOIN itemtype it
                        ON ei.itemtypecode = it.code
                INNER JOIN peak_cte pcte
                        ON pcte.BibNum = ed.BibNum
                           AND ed.ReportDate = pcte.ReportDate
         GROUP  BY ed.ReportDate,
                   ed.AuthorId)
SELECT TOP(100) ReportDate,
                a.Name,
                authorElementCount
FROM   allpeakauthorcount_cte apac
       INNER JOIN author a
               ON a.AuthorId = apac.AuthorId
ORDER  BY authorElementCount DESC,
          ReportDate,
          a.NAME 

In [ ]:
#!share --from sql-splSnapshots topPeakAuthorCount

using System.Linq;
using Microsoft.FSharp.Core;
using Plotly.NET;
using Plotly.NET.LayoutObjects;
using System;
using Range = Plotly.NET.StyleParam.Range;
using ChartDomain = Plotly.NET.ChartDomain.Chart;

var header =  new string[]{"Report Date", "Author Name", "Item Count"};
var topPeakAuthorTable = ChartDomain.Table<string, char, IEnumerable<string>,string>(
    header
    ,topPeakAuthorCount[0].Data.Select(x => new string[]{ ((DateTime)x["ReportDate"]).ToShortDateString(), x["Name"].ToString(), x["authorElementCount"].ToString()})
    ,MultiColumnWidth: new double[]{.15,.40,.15}
    ,CellsAlign: StyleParam.HorizontalAlign.Left
    
).WithLayout(
    Layout.init<int>(
        Width:475,
        Height:267,
        Margin:Margin.init<int,int,int,int,int,int>(Top:10, Left: 10, Right: 10, Bottom:10)
));


#!html
<a id="topPeakAuthorCount"></a>

#!csharp
topPeakAuthorTable


<!-- Plotly chart will be drawn inside this DIV --> 


 
</div

In [ ]:
#!sql-splSnapshots --name cookingCount


; WITH cookingsubject_cte
     AS (SELECT DISTINCT bibnum
         FROM   elementsubject
         WHERE  subjectid IN (SELECT subjectid
                              FROM   subject
                              WHERE  subject LIKE '%cooking%')),
     cookingnonfiction_cte
     AS (SELECT DISTINCT ei.bibnum
         FROM   elementinventory ei
                INNER JOIN cookingsubject_cte cs
                        ON cs.bibnum = ei.bibnum
                INNER JOIN itemcollection ic
                        ON ic.code = ei.itemcollectioncode
         WHERE  ic.categorygroup = 'nonfiction'),
     cookingsubjectcount_cte
     AS (SELECT reportdate,
                subjectid,
                Count(es.bibnum) ItemCount
         FROM   elementsubject es
                INNER JOIN cookingnonfiction_cte cn
                        ON cn.bibnum = es.bibnum
         GROUP  BY reportdate,
                   subjectid)
SELECT reportdate,
       itemcount,
       subject
FROM   cookingsubjectcount_cte csc
       INNER JOIN subject s
               ON csc.subjectid = s.subjectid
WHERE  itemcount > 100
ORDER  BY reportdate,
          itemcount DESC 

(455 rows affected)

reportdate,itemcount,subject
2017-01-01 00:00:00Z,496,Cooking
2017-01-01 00:00:00Z,199,Cooking American
2017-01-01 00:00:00Z,156,Vegan cooking
2017-01-01 00:00:00Z,156,Vegetarian cooking
2017-01-01 00:00:00Z,122,Quick and easy cooking
2017-01-01 00:00:00Z,116,Cooking Italian
2017-01-01 00:00:00Z,111,Cooking Natural foods
2017-02-01 00:00:00Z,497,Cooking
2017-02-01 00:00:00Z,200,Cooking American
2017-02-01 00:00:00Z,157,Vegan cooking


In [ ]:
#!share --from sql-splSnapshots cookingCount